# Πειραματισμοί

In [1]:
from numpy import *
from numpy.linalg import *
from filling import edge, create_edge, interpolate_color
from transforms_projections import *
import matplotlib.pyplot as plt
data = load('h3.npy', allow_pickle=True).tolist()
for (key, value) in data.items():
    exec(key + ' = value')
data.keys()

dict_keys(['verts', 'vertex_colors', 'face_indices', 'depth', 'cam_eye', 'cam_up', 'cam_lookat', 'ka', 'kd', 'ks', 'n', 'light_positions', 'light_intensities', 'Ia', 'M', 'N', 'W', 'H', 'bg_color'])

# Φωτισμός και Υλικό Επιφάνειας

## A1. Ambient Light

In [8]:
def ambient_light(ka, Ia):

    I = ka * Ia

    return I

## A2. Diffuse Light

In [2]:
def diffuse_light(P, N, color, kd, light_positions, light_intensities):

    I = zeros((3,))

    L = (light_positions - P) / norm(light_positions - P, axis=0)
    
    for i in range(light_positions.shape[1]):
        if N.T @ L[:, i] > 0:
            I += kd * light_intensities[:, i] * (N.T @ L[:, i])
    
    return I.reshape(-1, 1) * color
    

In [6]:
def specular_light(P, N, color, cam_pos, ks, n, light_positions, light_intensities):

    I = zeros((3,))

    L = (light_positions - P) / norm(light_positions - P, axis=0)
    R = (2 * N.T @ L) * N - L
    V = (cam_pos - P) / norm(cam_pos - P, axis=0)
    
    for i in range(light_positions.shape[1]):
        if V.T @ R[:, i] > 0 and N.T @ L[:, i] > 0:
            I += ks * light_intensities[:, i] * (V.T @ R[:, i]) ** n
    
    return I.reshape(-1, 1) * color

# B. Shading

## Β.1 Υπολογισμός κανονικών διανυσμάτων επιφάνειας

In [130]:
def calculate_normals(vertices, face_indices):

    normals = empty(shape=vertices.shape)

    surface_normals = cross(vertices[:, face_indices[1]] - vertices[:, face_indices[0]],
                            vertices[:, face_indices[2]] - vertices[:, face_indices[0]],
                            axis=0)

    surface_normals /= norm(surface_normals, axis=0)

    for i in range(vertices.shape[1]):
        S_Nk = sum(surface_normals[:, where(face_indices == i)[1]], axis=1).reshape(-1, 1)
        S_Nk /= norm(S_Nk, axis=0)
        normals[:, i] = S_Nk.reshape(-1)
    
    return normals

## Β2. Συνάρτηση φωτογράφισης

In [ ]:
verts2d, depth = project_cam_lookat(f=1)

In [127]:
def render_object(shader, focal, eye, lookat, up, bg_color, M, N, H, W, verts, vert_colors, face_indices, ka, kd, ks, n, light_positions, light_intensities, Ia):

    
    # 1. Calculate surface normals of all vertices of object
    normals = calculate_normals(verts, face_indices)
    
    # 2a. Change coordinate system based on where the camera is
    verts2d, depth = project_cam_lookat(f=focal,
                                        c_org=eye,
                                        c_lookat=lookat,
                                        c_up=up,
                                        verts3d=verts)

    # 2b. Project them in camera lens
    verts_rast = rasterize(verts2d=verts2d,
                           img_h=M,
                           img_w=N,
                           cam_h=H,
                           cam_w=W)
    
    vert_colors = flip(vert_colors, axis=1)
    vert_colors *= 255

    verts_rast = flip(verts_rast, axis=1)

    depth_of_triangles = mean(depth.reshape(-1)[face_indices.T], axis=1)
    priority_of_triangles = flip(argsort(depth_of_triangles))

    X = 255 *  ones((M, N, 3))
    
    for face in face_indices.T[priority_of_triangles]:

        if (all(verts_rast[face][0, :]) < W) and (all(verts_rast[face][1, :]) < H):
            X = shade_gouraud(verts_p=verts_rast[face],
                              verts_n=normals[face],
                              verts_c=vert_colors[face],
                              bcoords=sum(verts_rast[face], axis=0).reshape(-1, 1) / 3,
                              cam_pos=eye,
                              ka=ka,
                              kd=kd,
                              ks=ks,
                              n=n,
                              light_positions=light_positions,
                              light_intensities=light_intensities,
                              Ia=Ia,
                              X=X)
    
    plt.imshow(X)

## B3. Gouraud Shading

In [85]:
def shade_gouraud(verts_p, verts_n, verts_c, bcoords, cam_pos, ka, kd, ks, n, light_positions, light_intensities, Ia, X):

    for i in range(verts_c.shape[1]):

        verts_c[:, i] += ambient_light(ka, Ia) \
            + diffuse_light(P=bcoords,
                            N=verts_n[:, i],
                            color=verts_c[:, i],
                            kd=kd,
                            light_positions=light_positions,
                            light_intensities=light_intensities) \
                                + specular_light(P=bcoords,
                                                N=verts_n[:, i],
                                                color=verts_c[:, i],
                                                cam_pos=cam_pos,
                                                ks=ks,
                                                n=n,
                                                light_positions=light_positions,
                                                light_intensities=light_intensities)
        
    verts_pcn = hstack((verts_p.T, verts_c.T, verts_n.T))

    tuples_of_verts = array([delete(verts_pcn, i, axis=0) for i in range(3)])

    triangle_edges = fromiter((create_edge(tuples_of_verts[i]) for i in range(3)), dtype=edge)

    lowest_scanline = min(triangle_edges['y_min'])
    highest_scanline = max(triangle_edges['y_max'])

    active_edges = triangle_edges[(triangle_edges['y_min'] == lowest_scanline)]

    for y in range(lowest_scanline, highest_scanline):

        lower_edges = active_edges[active_edges['y_max'] == y]
        if lower_edges.size > 0:
            active_edges = delete(active_edges, active_edges == lower_edges)

        active_edges = sort(active_edges, order='intersect')

        leftmost_intersect = ceil(active_edges[0]['intersect'])
        rightmost_intersect = ceil(active_edges[1]['intersect'])

        Cl, Cr = interpolate_color(active_edges['y_max'],
                                   active_edges['y_min'],
                                   y,
                                   active_edges['RGB_max'],
                                   active_edges['RGB_min'])

        for x in range(int(leftmost_intersect), int(rightmost_intersect)):

            X[x, y] = interpolate_color(rightmost_intersect,
                                        leftmost_intersect,
                                        x,
                                        Cr,
                                        Cl)

        active_edges['intersect'] += 1 / active_edges['slope']

        upper_edges = triangle_edges[triangle_edges['y_min'] == y + 1]
        if upper_edges.size > 0:
            active_edges = append(active_edges, upper_edges)

    return X

## B4. Phong Shading

In [3]:
def shade_phong(verts_p, verts_n, verts_c, bcoords, cam_pos, ka, kd, ks, n, light_positions, light_intensities, Ia, X):
        
    verts_pcn = hstack((verts_p.T, verts_c.T, verts_n.T))

    tuples_of_verts = array([delete(verts_pcn, i, axis=0) for i in range(3)])

    triangle_edges = fromiter((create_edge(tuples_of_verts[i]) for i in range(3)), dtype=edge)

    lowest_scanline = min(triangle_edges['y_min'])
    highest_scanline = max(triangle_edges['y_max'])

    active_edges = triangle_edges[(triangle_edges['y_min'] == lowest_scanline)]

    for y in range(lowest_scanline, highest_scanline):

        lower_edges = active_edges[active_edges['y_max'] == y]
        if lower_edges.size > 0:
            active_edges = delete(active_edges, active_edges == lower_edges)

        active_edges = sort(active_edges, order='intersect')

        leftmost_intersect = ceil(active_edges[0]['intersect'])
        rightmost_intersect = ceil(active_edges[1]['intersect'])

        Cl, Cr = interpolate_color(active_edges['y_max'],
                                   active_edges['y_min'],
                                   y,
                                   active_edges['RGB_max'],
                                   active_edges['RGB_min'])

        Nl, Nr = interpolate_color(active_edges['y_max'],
                                   active_edges['y_min'],
                                   y,
                                   active_edges['N_max'],
                                   active_edges['N_min'])


        for x in range(int(leftmost_intersect), int(rightmost_intersect)):

            color = interpolate_color(rightmost_intersect,
                                      leftmost_intersect,
                                      x,
                                      Cr,
                                      Cl)
            
            normal = interpolate_color(rightmost_intersect,
                                      leftmost_intersect,
                                      x,
                                      Nr,
                                      Nl)

            color += ambient_light(ka, Ia) \
                + diffuse_light(P=bcoords,
                                N=normal,
                                color=color,
                                kd=kd,
                                light_positions=light_positions,
                                light_intensities=light_intensities) \
                                    + specular_light(P=bcoords,
                                                     N=normal,
                                                     color=color,
                                                     cam_pos=cam_pos,
                                                     ks=ks,
                                                     n=n,
                                                     light_positions=light_positions,
                                                     light_intensities=light_intensities)
            
        active_edges['intersect'] += 1 / active_edges['slope']

        upper_edges = triangle_edges[triangle_edges['y_min'] == y + 1]
        if upper_edges.size > 0:
            active_edges = append(active_edges, upper_edges)

    return X